In [1]:
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from IG import Integrated_Gradients
from PGD import PGD
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
from SmoothGrad import SmoothGrad

cuda = True if torch.cuda.is_available() else False
print(cuda)


preprocess = transforms.Compose([
    transforms.Resize((224,224)), # Resize with 224 
    transforms.ToTensor(), # making the matrix shape to [C,H,W] and range of [0,1]
])

norm = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))


val_imagefolder = ImageFolder(root='/workspace/my_data/imagenet/val', transform=preprocess)
val_dataloader = DataLoader(val_imagefolder, batch_size=1, shuffle=False)

TAIG_imagefolder = ImageFolder(root='/workspace/my_data/TAIG_test', transform=preprocess)
TAIG_dataloader = DataLoader(TAIG_imagefolder, batch_size=1, shuffle=False)


model = models.resnet50(pretrained=True)
model = nn.Sequential(norm, model)
model.eval() # batch가 1개일 때 가능하게함 Batchnorm, Dropout 사용안함


IG = Integrated_Gradients(model, cuda=cuda)
SG = SmoothGrad(model, cuda=cuda)
PGD = PGD(model, cuda=cuda)

for (ori_imgs, ori_labels), (TAIG_imgs,_) in zip(val_dataloader, TAIG_dataloader):
    
    print('=================================================================================================================')
    
    ori_predicted_idx, _, ori_SG = SG(ori_imgs)
    ori_SG = ori_SG.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    _, _, ori_IG = IG(ori_imgs)
    ori_IG = ori_IG.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    pgd_img = PGD(ori_imgs, ori_labels)
    _, pgd_prob, pgd_IG = IG(pgd_img, ori_predicted_idx)
    pgd_IG = pgd_IG.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    _,_, pgd_label_IG = IG(pgd_img)
    pgd_label_IG = pgd_label_IG.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    _,_, pgd_label_SG = SG(pgd_img, ori_predicted_idx)
    pgd_label_SG = pgd_label_SG.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    pgd_img = pgd_img.detach().cpu()
    pgd_noise = pgd_img-ori_imgs
    # pgd_noise = pgd_noise.sum(dim=1, keepdim=True)
    pgd_noise = (pgd_noise - torch.min(pgd_noise))/(torch.max(pgd_noise)-torch.min(pgd_noise))
    pgd_noise = pgd_noise.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    TAIG_noise = TAIG_imgs-ori_imgs
    # TAIG_noise = TAIG_noise.sum(dim=1, keepdim=True)
    TAIG_noise = (TAIG_noise - torch.min(TAIG_noise))/(torch.max(TAIG_noise)-torch.min(TAIG_noise))
    TAIG_noise = TAIG_noise.squeeze(0).detach().cpu().numpy().transpose(1,2,0)
    
    _, _, TAIG_label_SG = SG(TAIG_imgs)
    TAIG_label_SG = TAIG_label_SG.squeeze(0).detach().cpu().numpy().transpose(1,2,0)

    IG_pgd_noise_sim = abs(np.dot(ori_IG.reshape(-1), pgd_noise.reshape(-1)))/(np.linalg.norm(ori_IG.reshape(-1))*np.linalg.norm(pgd_noise.reshape(-1)))
    print(f'Similarity of noise between pgd_noise and pgd IG: {IG_pgd_noise_sim:.4f}')
    IG_pgd_label_noise_sim = abs(np.dot(pgd_label_IG.reshape(-1), pgd_noise.reshape(-1)))/(np.linalg.norm(pgd_label_IG.reshape(-1))*np.linalg.norm(pgd_noise.reshape(-1)))
    print(f'Similarity of noise between pgd_noise and pgd label IG: {IG_pgd_label_noise_sim:.4f}')
    
    SG_pgd_noise_sim = abs(np.dot(ori_SG.reshape(-1), pgd_noise.reshape(-1)))/(np.linalg.norm(ori_SG.reshape(-1))*np.linalg.norm(pgd_noise.reshape(-1)))
    print(f'Similarity of noise between pgd_noise and SG: {SG_pgd_noise_sim:.4f}')
    
    SG_pgd_label_noise_sim = abs(np.dot(pgd_label_SG.reshape(-1), pgd_noise.reshape(-1)))/(np.linalg.norm(pgd_label_SG.reshape(-1))*np.linalg.norm(pgd_noise.reshape(-1)))
    print(f'Similarity of noise between pgd_noise and pgd label SG: {SG_pgd_label_noise_sim:.4f}')
    
    SG_TAIG_noise_sim = abs(np.dot(ori_SG.reshape(-1), TAIG_noise.reshape(-1)))/(np.linalg.norm(ori_SG.reshape(-1))*np.linalg.norm(TAIG_noise.reshape(-1)))
    print(f'Similarity of noise between TAIG_noise and SG: {SG_TAIG_noise_sim:.4f}')
    
    SG_TAIG_label_noise_sim = abs(np.dot(TAIG_label_SG.reshape(-1), TAIG_noise.reshape(-1)))/(np.linalg.norm(TAIG_label_SG.reshape(-1))*np.linalg.norm(TAIG_noise.reshape(-1)))
    print(f'Similarity of noise between TAIG_noise and TAIG label SG: {SG_TAIG_label_noise_sim:.4f}')
    
    

True


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Similarity of noise between pgd_noise and pgd IG: 0.8254
Similarity of noise between pgd_noise and pgd label IG: 0.8275
Similarity of noise between pgd_noise and SG: 0.4987
Similarity of noise between pgd_noise and pgd label SG: 0.4936
Similarity of noise between TAIG_noise and SG: 0.5996
Similarity of noise between TAIG_noise and TAIG label SG: 0.6281
Similarity of noise between pgd_noise and pgd IG: 0.8221
Similarity of noise between pgd_noise and pgd label IG: 0.8256
Similarity of noise between pgd_noise and SG: 0.4060
Similarity of noise between pgd_noise and pgd label SG: 0.3895
Similarity of noise between TAIG_noise and SG: 0.4912
Similarity of noise between TAIG_noise and TAIG label SG: 0.6440
Similarity of noise between pgd_noise and pgd IG: 0.8211
Similarity of noise between pgd_noise and pgd label IG: 0.8187
Similarity of noise between pgd_noise and SG: 0.4647
Similarity of noise between pgd_noise and pgd label SG: 0.4479
Similarity of noise between TAIG_noise and SG: 0.5663
